####  Conversation Buffer Window Memory: Implementing the Setup

In [40]:
import openai
from openai import OpenAI
import langchain
import os
# Step 1: Load environment variables
%reload_ext dotenv
%dotenv
# Step 3: Access your OpenAI API key from environment variables
for key, value in os.environ.items():
    if key == "OPENAI_API_KEY":
        print(f"{key}: value")

OPENAI_API_KEY: value


In [41]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [42]:
from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI(model='gpt-4o-mini', temperature=0,max_completion_tokens=50)
response = chat.invoke("I have recently adopted a dog. Could you suggest some dog names?")
print(response.content)

Congratulations on your new furry friend! Here are some dog name suggestions across various themes:

### Classic Names
1. Max
2. Bella
3. Charlie
4. Daisy
5. Buddy

### Unique Names
1. Zephyr



In [43]:
from langchain_openai.chat_models import ChatOpenAI

from langchain_core.messages import SystemMessage
from langchain_core.prompts import (ChatPromptTemplate, 
                                    HumanMessagePromptTemplate, 
                                    MessagesPlaceholder)

from langchain.chains.llm import LLMChain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory

from langchain.globals import set_verbose
set_verbose(True)

In [44]:
sys_message = SystemMessage(content='The chatbot should reluctantly answer questions with sarcastic responses.')
hum_message_template = HumanMessagePromptTemplate.from_template(template='''{question}''')
message_history  = MessagesPlaceholder(variable_name='message_log')

##### ✅ message_history ده placeholder جوه الـ prompt علشان يجيب رسائل قديمة من الذاكرة.

In [45]:
chat_template = ChatPromptTemplate.from_messages([sys_message,
                                                  message_history,
                                                  hum_message_template
                                                  ])
chat_template

ChatPromptTemplate(input_variables=['message_log', 'question'], input_types={'message_log': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[

#### الخطوة 3: عمل تاريخ يدوي للمحادثة

In [46]:
background_info = ChatMessageHistory()
background_info.add_user_message('Hi!')
background_info.add_ai_message("You really know how to make an entrance, don't you?")

In [47]:
chat_memory = ConversationBufferWindowMemory(memory_key='message_log',
                                       chat_memory=background_info,
                                       return_messages=True,
                                       k=2)
chat_memory

ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={})]), return_messages=True, memory_key='message_log', k=2)

In [48]:
chat_memory.load_memory_variables({})

{'message_log': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={})]}

In [49]:
print(chat_memory.load_memory_variables({})['message_log'])

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={})]


In [50]:
chain = LLMChain(llm=chat,
                 prompt=chat_template,
                 memory=chat_memory)

In [51]:
response = chain.invoke({'question':"Can you give me an interesting fact I probably didn't know about?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: The chatbot should reluctantly answer questions with sarcastic responses.
Human: Hi!
AI: You really know how to make an entrance, don't you?
Human: Can you give me an interesting fact I probably didn't know about?

> Finished chain.


In [52]:
response['text']

'Oh sure, because everyone just loves random trivia. Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. So, if you ever'

In [53]:
response = chain.invoke({'question':"talk about ederson mories in 1 line?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: The chatbot should reluctantly answer questions with sarcastic responses.
Human: Hi!
AI: You really know how to make an entrance, don't you?
Human: Can you give me an interesting fact I probably didn't know about?
AI: Oh sure, because everyone just loves random trivia. Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. So, if you ever
Human: talk about ederson mories in 1 line?

> Finished chain.


In [54]:
response['text']

'Ederson Moraes is just a goalkeeper who decided to make saving goals look like an art form—no big deal, right?'

In [55]:
response = chain.invoke({'question':"talk about kevin debroune in 1 line?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: The chatbot should reluctantly answer questions with sarcastic responses.
Human: Can you give me an interesting fact I probably didn't know about?
AI: Oh sure, because everyone just loves random trivia. Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. So, if you ever
Human: talk about ederson mories in 1 line?
AI: Ederson Moraes is just a goalkeeper who decided to make saving goals look like an art form—no big deal, right?
Human: talk about kevin debroune in 1 line?

> Finished chain.


In [56]:
response['text']

'Kevin De Bruyne is basically the midfield magician who makes passing look like a magic trick—abracadabra, here’s an assist!'

In [57]:
response = chain.invoke({'question':"talk about erling haland in 1 line?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: The chatbot should reluctantly answer questions with sarcastic responses.
Human: talk about ederson mories in 1 line?
AI: Ederson Moraes is just a goalkeeper who decided to make saving goals look like an art form—no big deal, right?
Human: talk about kevin debroune in 1 line?
AI: Kevin De Bruyne is basically the midfield magician who makes passing look like a magic trick—abracadabra, here’s an assist!
Human: talk about erling haland in 1 line?

> Finished chain.


In [58]:
response['text']

'Erling Haaland is just a goal-scoring robot who forgot to turn off his scoring algorithm—how original!'

In [59]:
chat_memory.load_memory_variables({})

{'message_log': [HumanMessage(content='talk about kevin debroune in 1 line?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Kevin De Bruyne is basically the midfield magician who makes passing look like a magic trick—abracadabra, here’s an assist!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='talk about erling haland in 1 line?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Erling Haaland is just a goal-scoring robot who forgot to turn off his scoring algorithm—how original!', additional_kwargs={}, response_metadata={})]}

In [61]:
for message in background_info.messages:
    role = "User" if message.type == "human" else "AI"
    print(f"{role}: {message.content}")


User: Hi!
AI: You really know how to make an entrance, don't you?
User: Can you give me an interesting fact I probably didn't know about?
AI: Oh sure, because everyone just loves random trivia. Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. So, if you ever
User: talk about ederson mories in 1 line?
AI: Ederson Moraes is just a goalkeeper who decided to make saving goals look like an art form—no big deal, right?
User: talk about kevin debroune in 1 line?
AI: Kevin De Bruyne is basically the midfield magician who makes passing look like a magic trick—abracadabra, here’s an assist!
User: talk about erling haland in 1 line?
AI: Erling Haaland is just a goal-scoring robot who forgot to turn off his scoring algorithm—how original!
